In [34]:
import json
import os.path as osp
import glob
import cv2
import numpy as np
import glob
from multiprocessing import Pool
from tqdm import tqdm
from pathlib import Path
import shutil
import pandas as pd
from PIL import Image

# rotate setup

In [35]:
# predict by image align (before ptmap)
ROTATE = 270 # 逆时针
img_align_result = f"/data/home/yanghanlong/results/image_align/ptmap_5k/my_model/rotate/rotate_{ROTATE}/v3_5_8_epoch9"
# img_align_result = "/data/home/yanghanlong/results/image_align/my_model/ptmap_5k/my_model/v2_2_epoch3_remove_isopt_4"
# img_align_result = "/data/home/yanghanlong/results/image_align/original_model/ptmap_5k_with_rtps_params"

## output dir after ptmap
output_after_ptmap_dir = f"/data/home/yanghanlong/results/pt_map/eval_mymodel_image_align/rotate_{ROTATE}/v3_5_8_epoch9"
# output_after_ptmap_dir = "/data/home/yanghanlong/results/pt_map/eval_mymodel_image_align/original_model/ptmap5k"

## file path of template pts coord from <data/image_registration/anno/imageid.json>
tem_pts_coord_dir = "/data/home/yanghanlong/data/image_registration/anno"

## image path to get w, h
image_dir = f"/data/home/yanghanlong/data/image_retrieval/ptmap_5k_rotate/{ROTATE}"

expanded = False # no use; maybe discarded
## make dir
Path(f"{output_after_ptmap_dir}/results").mkdir(parents=True,exist_ok=True)

# normal setup

In [36]:
# # predict by image align (before ptmap)
# ROTATE = 0 # 逆时针
# img_align_result = "/data/home/yanghanlong/results/image_align/ptmap_5k/my_model/rotate/rotate_0/v3_5_8_epoch9"
# # img_align_result = "/data/home/yanghanlong/results/image_align/ptmap_5k/my_model/v0811_delete_selfmatchpt" # 线上
# # img_align_result = "/data/home/yanghanlong/results/image_align/original_model/ptmap_5k_with_rtps_params"

# ## output dir after ptmap
# output_after_ptmap_dir = "/data/home/yanghanlong/results/pt_map/eval_mymodel_image_align/v3_5_8_epoch9"
# # output_after_ptmap_dir = "/data/home/yanghanlong/results/pt_map/eval_mymodel_image_align/original_model/ptmap5k"

# ## file path of template pts coord from <data/image_registration/anno/imageid.json>
# tem_pts_coord_dir = "/data/home/yanghanlong/data/image_registration/anno"

# ## image path to get w, h
# image_dir = f"/data/home/yanghanlong/data/image_retrieval/image"

# expanded = False
# ## make dir
# Path(f"{output_after_ptmap_dir}/results").mkdir(parents=True,exist_ok=True)

# PTMAP function

In [37]:
def transform_template_pt(template_pt, H, tps_weights, template_mkpts_tps_,
                            parallex, overlap, K_smooth = 5):
    '''
    Args:
        template_pt: ndarray, (2, ),
        template_mkpts_tps_: ndarray, (2, n),
        parallex: ndarray, (2, n)
    '''
    ## Do global homographic transform
    template_hpt = np.append(template_pt, 1) # (3, )
    # import pdb
    # pdb.set_trace()
    template_hpt_ = H @ template_hpt # (3, )
    template_pt_ = template_hpt_ / (template_hpt_[2] + 1e-8)
    u_, v_ = template_pt_[:2]

    ## Do tps transformation
    gx, hy = 0, 0
    offset_u0_, offset_u1_ = overlap['offset_u0_'], overlap['offset_u1_']
    offset_v0_, offset_v1_ = overlap['offset_v0_'], overlap['offset_v1_']
    if (offset_u0_ <= u_ <= offset_u1_) and (offset_v0_ <= v_ <= offset_v1_):
        x1_, y1_ = template_mkpts_tps_
        n = len(x1_)
        gx_sub, hy_sub = 0, 0
        wx, wy = tps_weights['wx'], tps_weights['wy']
        a, b = tps_weights['a'], tps_weights['b']
        for kf in range(n):
            dist2 = (u_ - x1_[kf]) ** 2 + (v_ - y1_[kf]) ** 2
            rbf = 0.5 * dist2 * np.log(dist2)
            gx_sub += wx[kf] * rbf
            hy_sub += wy[kf] * rbf

        gx_sub += a[0] * u_ + a[1] * v_ + a[2]
        hy_sub += b[0] * u_ + b[1] * v_ + b[2]
        gx, hy = gx_sub, hy_sub

    ## smooth transition to global transform
    gxn, hyn = parallex
    eta_d0 = 0
    eta_d1 = K_smooth * max(abs(np.concatenate([gxn, hyn])))
    sub_u0_, sub_u1_ = overlap['sub_u0_'], overlap['sub_u1_']
    sub_v0_, sub_v1_ = overlap['sub_v0_'], overlap['sub_v1_']
    sub_u0_ = sub_u0_ + min(gxn)
    sub_u1_ = sub_u1_ + max(gxn)
    sub_v0_ = sub_v0_ + min(hyn)
    sub_v1_ = sub_v1_ + max(hyn)
    dist_horizontal = np.maximum(sub_u0_-u_, u_-sub_u1_)
    dist_vertical = np.maximum(sub_v0_-v_, v_-sub_v1_)
    dist_sub = np.maximum(dist_horizontal, dist_vertical)
    dist_sub = np.maximum(0, dist_sub)
    eta = (eta_d1 - dist_sub) / (eta_d1 - eta_d0)
    # eta[dist_sub < eta_d0] = 1
    # eta[dist_sub > eta_d1] = 0
    if dist_sub < eta_d0:
        eta = 1
    elif dist_sub > eta_d1:
        eta = 0

    gx *= eta
    hy *= eta

    res_u_ = u_ - gx
    res_v_ = v_ - hy
    return [res_u_, res_v_]

In [38]:
def find_gt_index(data):
    for k in data["alignment_results"]:
        if data["alignment_results"][k]["isGroundTruthTemplate"] == 1:
            return k
    return -1

def find_mpt(tem_pt, alignment_params):

    H, tps_weights, template_mkpts_tps_,parallex, overlap = alignment_params["H"],alignment_params["tps_weights"],alignment_params["template_mkpts_tps_"],alignment_params["parallex"],alignment_params["overlap"]
    sam_pt = transform_template_pt(tem_pt, H, tps_weights, template_mkpts_tps_,parallex, overlap, K_smooth = 5)
    return (tem_pt, sam_pt) # ([x1,x2],[y1,y2])

def pt_scale2ori( mpts,data, k):
    tem_scale = data["alignment_results"][k]["alignment_params"]["template_xy_scales"]
    sam_scale = data["alignment_results"][k]["alignment_params"]["sample_xy_scales"]
    _mpts = []
    for pt in mpts:
        pt[0][0] /= tem_scale[0]
        pt[0][1] /= tem_scale[1]
        pt[1][0] /= sam_scale[0]
        pt[1][1] /= sam_scale[1]
        _mpts.append(pt)
    return _mpts

def sam_pts_rotate(mpts, data, k, rotate):
    h,w,_ = data["alignment_results"][k]["sample_features"]["sam_shape"]
    if rotate == 90:
        _mpts = []
        for pt in mpts:
            pt[1][0],pt[1][1] = h-pt[1][1],pt[1][0]
            _mpts.append(pt)
    elif rotate == 180:
        _mpts = []
        for pt in mpts:
            pt[1][0],pt[1][1] = w-pt[1][0],h-pt[1][1]
            _mpts.append(pt)
    elif rotate == 270:
        _mpts = []
        for pt in mpts:
            pt[1][0],pt[1][1] = pt[1][1],w-pt[1][0]
            _mpts.append(pt)
    elif rotate == 0:
        _mpts = mpts
    return _mpts
    
def find_tem_pts(alignment_params,img_id,tem_pts_coord_dir,expanded=False,template_w=0):
    with open(f"{tem_pts_coord_dir}/{img_id}.json",'r') as f:
        template_pts = json.load(f)["template_pts"]

    ## --- HxW -> resized HxW ---
    tem_scale = alignment_params["template_xy_scales"]
    sam_scale = alignment_params["sample_xy_scales"]
    for i in range(len(template_pts)):
        if expanded:
            template_pts[i][0] += (0.15/1.3*template_w)
        template_pts[i][0] *= tem_scale[0]
        template_pts[i][1] *= tem_scale[1]

    return [find_mpt(x, alignment_params) for x in template_pts]
    

def job(js):
    with open(js, 'r') as f:
        data = json.load(f)
    im_id = data["id"]
    # find gt alignment results
    gt_index = find_gt_index(data)
    if gt_index == -1:
        with open(f"{output_after_ptmap_dir}/no_gt_id.txt", 'a') as f:
            f.write('\n')
            f.write(im_id)
        return (im_id,None,None)
    else:
        alignment_params = data["alignment_results"][gt_index]["alignment_params"]
        template_path = data["alignment_results"][gt_index]["template_path"]
    # find template pts(generally 16) and scale to (480,640) and map for each pt pair
    # template_w = data["alignment_results"][gt_index]["template_features"]["tem_shape"][1]
    template_w = 0
    ratio = data["alignment_results"][gt_index]["matched_features"]["mkpts_ratio2template"]
    ratio_modify_by_selfmatch = data["alignment_results"][gt_index]["matched_features"]['ratio_modify_by_image0_0']
    mpts = find_tem_pts(alignment_params,im_id,tem_pts_coord_dir,expanded=expanded,template_w=template_w)

    mpts = pt_scale2ori(mpts, data, gt_index)
    mpts = sam_pts_rotate(mpts, data, gt_index, rotate=ROTATE)
    template_pts = [[x[0][0], x[0][1]]for x in mpts]
    sample_pts = [[x[1][0], x[1][1]]for x in mpts]
    o_data = {
        "image_id": im_id,
        "template_path":template_path,
        "num_mpts": len(template_pts),
        "template_pts": template_pts,
        "sample_pts": sample_pts
    }
    
    with open(f"{output_after_ptmap_dir}/results/{im_id}.json", 'w') as f:
        json.dump(o_data, f)
    return (im_id,ratio,ratio_modify_by_selfmatch)



# Calculate distance function

In [39]:
##GroundTruth
gt_anno_dir = "/data/home/yanghanlong/data/image_registration/anno"
##unmatched case(like template update)
unmatched_csv = "/data/home/yanghanlong/data/image_registration/image_id.unmatched.csv"

##return a list of distance of each pair of pts
##modified distance, divided by sample area

def calculate_distance3(image_id):
    pred_anno_dir = f"{output_after_ptmap_dir}/results"
    pre_anno = json.load(open(Path(pred_anno_dir).joinpath(image_id+'.json'), 'r'))
    gt_anno = json.load(open(Path(gt_anno_dir).joinpath(image_id+'.json'), 'r'))
    pre_template_pts = pre_anno["template_pts"]
    pre_sample_pts = pre_anno["sample_pts"]
    gt_template_pts = gt_anno["template_pts"]
    gt_sample_pts = gt_anno["sample_pts"]
    ##ignore [Null, Null] in gt_sample_pts
    ignore_pt = [None, None]
    while ignore_pt in gt_sample_pts:
        index = gt_sample_pts.index(ignore_pt)
        del gt_sample_pts[index]
        del pre_sample_pts[index]
    ##transform to np array
    pre_sample_pts = np.array(pre_sample_pts)
    gt_sample_pts = np.array(gt_sample_pts)
    ##calculate distance
    distance = [np.linalg.norm(x - y) for x,y in zip(gt_sample_pts, pre_sample_pts)]
    ##modify
    im = Image.open(f"{image_dir}/{image_id}.jpg")
    w, h = im.size
    assert w != 0
    distance = [ x / (np.sqrt(w*h)*10e-4) for x in distance]
    return distance

# debug

In [40]:
# j = glob.glob(f"{img_align_result}/*.json")[0]
# print(j)
# job(j)

# generate ptmap and ratio_list

In [41]:
## generate ptmap
jpath_list = glob.glob(f"{img_align_result}/*.json")
print("="*30)
print(f"Total file num: {len(jpath_list)}")                
print("="*30)
with Pool(30) as p:
    ratio_list = p.map(job, tqdm(jpath_list))
ratio_list = [x for x in ratio_list if x[1] != None]
#print results info
suc =  glob.glob(f"{output_after_ptmap_dir}/results/*.json")
print("="*30)
print(f"Successful file num: {len(suc)}")  
print("="*30)

Total file num: 4993


  0%|          | 0/4993 [00:00<?, ?it/s]

100%|██████████| 4993/4993 [00:02<00:00, 1665.79it/s]


Successful file num: 4636


# calculate distance

In [42]:
## calculate distance (pred vs. GT)
##handle all files by handle 1 file, choose calculate_distance2/calculate_distance
#unmatched id list
unmatched_list = pd.read_csv(unmatched_csv,header=None)[0].tolist()
jlist = glob.glob(output_after_ptmap_dir+"/results/*.json")
print(f"Totally {len(jlist)} cases to handle.")
dist_list = []
dist_dict = {}
for jterm in tqdm(jlist):
    image_id = Path(jterm).stem
    #skip unmatched id
    if image_id in unmatched_list:
        continue
    dist = calculate_distance3(image_id) ###CHECK
    dist_list.append(dist)
    dist_dict[image_id] = dist
print(f"finally {len(dist_list)} cases.")

Totally 4636 cases to handle.


  0%|          | 0/4636 [00:00<?, ?it/s]

100%|██████████| 4636/4636 [00:01<00:00, 3454.20it/s]

finally 4616 cases.


In [43]:
##for distance3(list)
count = sum([len(x) for x in dist_list])
total = sum([sum(x) for x in dist_list])
print(f"Average distance of all pt pairs: {total/count:.3f}")
recall_list = []
for threshold in range(10, 60,10):
    suc_num = sum([(np.array(x) < threshold).sum() for x in dist_list])
    if threshold == 10:
        print(f"Recall@{threshold}: {suc_num/count:.3%}")
    recall_list.append(suc_num/count)
print(f"AR@10:50:10: {sum(recall_list)/len(recall_list):.3%}")
# average ratio
pure_ratio_list = [x[1] for x in ratio_list]
print(f"Average ratio: {sum(pure_ratio_list)/len(pure_ratio_list):.6f}\tAverage ratio_modify_by_selfmatch: {sum([x[2] for x in ratio_list])/len(ratio_list):.6f}")
print(f"Cases(ratio < 0.2): {len([x for x in pure_ratio_list if x < 0.2])}\tCases(ratio_modify_by_selfmatch < 0.2): {len([x for x in ratio_list if x[2] < 0.2])}")
print(f"Cases(ratio < 0.1): {len([x for x in pure_ratio_list if x < 0.1])}\tCases(ratio_modify_by_selfmatch < 0.1): {len([x for x in ratio_list if x[2] < 0.1])}")
print(f"Cases (ratio < 0.2) example: {[x for x in ratio_list if x[1] < 0.2][:10]}")

Average distance of all pt pairs: 3.744
Recall@10: 95.254%
AR@10:50:10: 97.226%
Average ratio: 0.650122	Average ratio_modify_by_selfmatch: 0.650122
Cases(ratio < 0.2): 226	Cases(ratio_modify_by_selfmatch < 0.2): 226
Cases(ratio < 0.1): 130	Cases(ratio_modify_by_selfmatch < 0.1): 130
Cases (ratio < 0.2) example: [('03e7b3c1-c3f2-4800-b192-bfcfb097a7a9', 0.17410714285714285, 0.17410714285714285), ('08bdf88a-eb6a-4c43-863c-584d3077bdc9', 0.020330368487928845, 0.020330368487928845), ('0866b016-3b7f-4c96-baaf-2a2da554b16d', 0.09788359788359788, 0.09788359788359788), ('0eab6054-e056-4e61-9158-5ef194485e4e', 0.1953125, 0.1953125), ('0e5f4039-d0cf-4969-bd02-28346d627c58', 0.091796875, 0.091796875), ('0fee9d92-42ec-4d10-9ba3-132ad0fb277a', 0.033203125, 0.033203125), ('069ef367-0fd2-418e-8dd9-a8c6967f0934', 0.177734375, 0.177734375), ('0ce6f5f9-a3a7-4ba8-92e7-d9a7e0d00b11', 0.13145539906103287, 0.13145539906103287), ('023ae7de-85af-4c27-9c4f-aed35a3b1b62', 0.1038961038961039, 0.1038961038961039)

# check aver value of distance

In [44]:
aver_thres = 10

dist_aver_list = [sum(x)/len(x) for x in dist_list]
print(f"Success ratio(aver dist < {aver_thres}): {len([x for x in dist_aver_list if x < aver_thres])/len(jpath_list):.1%}")
dist_aver_dict = {k:(sum(v)/len(v)) for k,v in dist_dict.items()}
max_key = sorted(dist_aver_dict.keys(), key=dist_aver_dict.get,reverse=True)
for i in range(5):
    print(f"{max_key[i]} {dist_aver_dict[max_key[i]]}")

Success ratio(aver dist < 10): 86.5%
090f8be4-54c8-456b-90dc-bd6d6155a902 387.7851613626865
0ca3fe96-1a92-4ad4-87f1-0abd548762dc 302.4453143566294
08cf4d41-faa8-45c7-b2f0-a3cb6bf28add 226.8563450932498
0ca3264b-f603-4a0e-b69d-39e12ee42cf4 196.1612742058414
0a74bc93-0e37-40a1-950d-2bf72844f47b 194.80233754450484


# check max value of distance

In [97]:
dist_sum_dict = {k:sum(v) for k,v in dist_dict.items()}
max_key = sorted(dist_sum_dict.keys(), key=dist_sum_dict.get,reverse=True)
for i in range(5):
    print(f"{max_key[i]} {dist_sum_dict[max_key[i]]}")


01a332e3-445b-4e64-8d84-e5e19c9ac8b2 8304.688879009067
0e14e5d6-a38e-4efc-87b0-88d7ff682dec 7980.670276769891
0c086640-689c-4070-acdf-07019455e1b5 4372.753874706445
0f3ce993-cd5b-4fd5-affd-133d16996f9b 3993.8934128345895
0fb387db-c1bc-451a-8a57-d7a001065d67 2145.3819529456996


## save all results

In [22]:
results_p = "/data/home/yanghanlong/mymodel_v0811_2048_img_align_ptmap5k_results.csv"


dist_sum_list = [dist_sum_dict[x] for x in max_key]
o = pd.DataFrame({'image_id':max_key,'distance_sum':dist_sum_list})
o.to_csv(results_p,index=False,sep='\t')

## copy [blended, sample, matches] in bad case to one folder **(need run max_key first)**

In [98]:
temp_chech_dir = Path("/data/home/yanghanlong/temp/v3_5_6_epoch14")
copy_num = 20

dist_sum_dict = {k:sum(v) for k,v in dist_dict.items()}
max_key = sorted(dist_sum_dict.keys(), key=dist_sum_dict.get,reverse=True)
def copyImgAlignResults(image_id_list,output_dir,image_align_results_dir=None):
    if image_align_results_dir == None:
        image_align_results_dir = img_align_result
    for image_id in image_id_list:
        shutil.copytree(f"{img_align_result}/{image_id}",Path(output_dir) / image_id)

# temp_chech_dir = Path("/data/home/yanghanlong/temp/original_model")
temp_chech_dir.mkdir(exist_ok=True)
copyImgAlignResults(max_key[0:copy_num],temp_chech_dir)
# copyImgAlignResults(aa,temp_chech_dir)

In [55]:
dt = pd.read_csv('/data/home/yanghanlong/compare.csv',sep='\t')
# 排序dt["diff[(_nodel)-(_del)]"]列中负数中绝对值最大的20个image_id
dt = dt.sort_values(by="diff[(_nodel)-(_del)]",ascending=True)
dt

,image_id,distance_sum_nodel,distance_sum_del,diff[(_nodel)-(_del)]
0,022d022b-e9d0-4132-8188-769a311784a0,5.177852,342.397757,-337.219905
2,0163d0ff-f079-4267-bd12-146e3cfd6bc0,9.673700,116.359219,-106.685519
3,0e71b233-7d73-45c9-bec4-2959bd7394f7,17.705952,121.882608,-104.176656
4,0adbe9f9-9bce-4988-9954-761136a5e32a,9.641336,22.222624,-12.581288
5,0cfd205a-c30f-4dba-9088-ab56e22d7ea2,4.688163,15.448004,-10.759842
...,...,...,...,...
10,0b48ea64-6d3f-453c-ae7f-2f9003988201,15.411926,11.663885,3.748041
9,0087e504-8558-4144-88c5-9b24ba28dc1d,49.909473,45.200406,4.709067
7,02fd9ec8-f8b1-44b1-b295-e4421867de04,61.058938,55.022899,6.036038
6,098924c3-6dbc-4de6-878f-1ba7c410f947,12.386085,6.252266,6.133819


## compare two csv results. 

In [23]:
csv1="/data/home/yanghanlong/mymodel_v0811_2048_img_align_ptmap5k_results.csv"
csv2="/data/home/yanghanlong/mymodel_v0811_2048_del_img_align_ptmap5k_results.csv"
compare_output = "/data/home/yanghanlong/compare.csv"
suffixes = ('_nodel','_del')

df1 = pd.read_csv(csv1,sep='\t')
df2 = pd.read_csv(csv2,sep='\t')
# merge and diff
merge=pd.merge(df1,df2,on='image_id',suffixes=suffixes)
diff_col_name = f'diff[({suffixes[0]})-({suffixes[1]})]'
merge[diff_col_name] = merge[f"distance_sum{suffixes[0]}"] - merge[f"distance_sum{suffixes[1]}"]
# sort
sorted_merge = merge.sort_values(by=diff_col_name, key=lambda x: abs(x), ascending=False)
sorted_merge.to_csv(compare_output,sep='\t',index=False)

In [24]:
print('---- positive means better, negative means worse')
print(f"diff postive num: {(sorted_merge[diff_col_name] > 0).sum()}")
print(f"diff negative num: {(sorted_merge[diff_col_name] < 0).sum()}")
print(f"diff > 10 num: {(sorted_merge[diff_col_name] > 10).sum()}")
print(f"diff < 10 num: {(sorted_merge[diff_col_name] < -10).sum()}")


---- positive means better, negative means worse
diff postive num: 284
diff negative num: 299
diff > 10 num: 1
diff < 10 num: 5
